# 미디어파이프 핸드(포즈x)를 이용해 진행

## ✅ 1단계: 전체 파이프라인 스켈레톤 코드

이 코드는 비디오를 읽고 → 프레임 단위로 손을 추적하고 → 후속 처리를 위한 구조를 잡아줍니다.
아직 세부 기능(홀드 검출, 시간 분석 등)은 비워두고 “틀”만 먼저 구성합니다.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe Hands 설정
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 비디오 열기
video_path = "climbing_video.mp4"  # ← 여기에 업로드한 영상 경로 넣기
cap = cv2.VideoCapture(video_path)

# 프레임 속도 확인
fps = cap.get(cv2.CAP_PROP_FPS)

# 프레임 카운터
frame_index = 0

# 홀드 분석 기록용 데이터 구조 (예: hold_logs["Hold A"] = [10,11,12,...])
hold_logs = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 이미지 전처리
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손 위치 분석
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크 시각화
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # 엄지끝 위치 (예시)
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            h, w, _ = frame.shape
            x_px, y_px = int(thumb_tip.x * w), int(thumb_tip.y * h)

            # ✅ STEP 2에서 이 손 위치와 자동 감지된 홀드가 겹치는지 확인할 예정
            # if is_inside_hold(x_px, y_px, hold_regions):
            #     hold_logs["Hold A"].append(frame_index) ...

    # 결과 보기
    cv2.imshow("Climbing Pose", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_index += 1

cap.release()
cv2.destroyAllWindows()

## 🧱 설명 요약

| 구성 요소         | 역할                                           |
|------------------|------------------------------------------------|
| `VideoCapture`   | 비디오에서 프레임을 추출                       |
| `MediaPipe Hands`| 손의 좌표(랜드마크)를 추출                     |
| `frame_index`    | 어떤 프레임에서 어떤 홀드를 잡았는지 기록용    |
| `hold_logs`      | 홀드별로 손이 닿은 프레임들을 저장하는 딕셔너리 |

## ✅ 2단계 목표

각 프레임에서 **홀드로 보이는 영역(색/형태 기반)** 을 자동으로 탐지해서
손 위치와 겹치는지 판단할 수 있도록 hold_regions를 만듭니다.

 🔍 기본 전략

1. 프레임을 **HSV 색상공간**으로 변환
2. 특정 색 범위에 해당하는 부분만 **마스크(mask)** 생성
3. `findContours`로 **윤곽 검출 → 사각형으로 추출**
4. 후보 홀드들을 **`hold_regions` 리스트**로 정리

In [ ]:
def detect_holds(frame_bgr):
    """입력 프레임에서 홀드로 추정되는 영역을 반환합니다"""
    hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)

    # 🎯 예시: 주황색 홀드를 탐지 (HSV 범위는 조정 가능)
    lower_orange = np.array([5, 100, 100])
    upper_orange = np.array([20, 255, 255])
    mask = cv2.inRange(hsv, lower_orange, upper_orange)

    # 잡음을 줄이기 위한 블러 + 팽창/침식
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations=2)

    # 윤곽선 검출
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 검출된 홀드 후보들을 사각형 영역으로 변환
    hold_regions = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500:  # 너무 작은 것은 제외 (노이즈 제거)
            x, y, w, h = cv2.boundingRect(cnt)
            hold_regions.append({"x": x, "y": y, "w": w, "h": h})

    return hold_regions

In [ ]:
# STEP 2: 홀드 감지
hold_regions = detect_holds(frame)

# 시각화 (사각형 표시)
for hold in hold_regions:
    cv2.rectangle(frame, (hold["x"], hold["y"]),
                  (hold["x"] + hold["w"], hold["y"] + hold["h"]),
                  (0, 255, 255), 2)
    cv2.putText(frame, "Hold", (hold["x"], hold["y"] - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

* 1단계 파이프라인에 추가하기

### 📌 조정 가능한 요소

| 항목              | 설명                                                                 |
|-------------------|----------------------------------------------------------------------|
| **HSV 색 범위**    | `lower_orange`, `upper_orange` — 영상에 맞춰 동적으로 조정 필요         |
| **최소 윤곽 크기** | `area > 500` — 너무 작은 물체(노이즈)는 제외                            |
| **색이 다양할 경우** | 여러 색상 범위에 대해 반복 적용하거나, `KMeans` 색상 클러스터링으로 자동 추출 가능 |

##  🎯 3단계 목표

손이 **특정 홀드 안에 들어간 프레임들**을 모아 다음을 분석합니다:

- 👉 **몇 번 잡았는지**
- 👉 **얼마나 오래 잡았는지** (초 단위 기준)

In [ ]:
"""
hold_logs = {
    "Hold_100_250": [12, 13, 14, 20, 21],
    "Hold_300_120": [33, 34, 35, 70, 71, 72]
}
이전단계에서 홀드 감지 후, 각 홀드에 대해 손이 닿은 프레임 인덱스를 기록합니다.
이후 이 hold_logs를 사용하여 각 홀드에 대한 통계나 피드백을 제공할 수 있습니다.
"""

import pandas as pd

def analyze_hold_times(hold_logs, fps):
    """
    각 홀드별로 몇 초 동안 잡았는지 분석합니다.
    연속된 프레임을 '한 번 잡은 동작'으로 간주합니다.
    """
    results = []

    for hold_id, frames in hold_logs.items():
        frames = sorted(frames)
        segments = []
        segment = [frames[0]]

        for i in range(1, len(frames)):
            if frames[i] == frames[i - 1] + 1:
                segment.append(frames[i])
            else:
                segments.append(segment)
                segment = [frames[i]]

        segments.append(segment)

        for seg in segments:
            start_f = seg[0]
            end_f = seg[-1]
            duration = (end_f - start_f + 1) / fps  # 초 단위로 변환
            results.append({
                "Hold ID": hold_id,
                "Start Frame": start_f,
                "End Frame": end_f,
                "Duration (s)": round(duration, 2)
            })

    return pd.DataFrame(results)

In [ ]:
# 시각화 코드 ( 선택 )

import matplotlib.pyplot as plt

def plot_hold_times(df):
    df.groupby("Hold ID")["Duration (s)"].sum().plot(kind="bar")
    plt.title("홀드별 누적 잡은 시간 (초)")
    plt.ylabel("초")
    plt.xlabel("홀드 ID")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 1,2,3 단계 코드 통합하기

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

# 경로 설정
video_path = "/Users/laxdin24/Downloads/IMG_6202.MP4"  # 분석할 비디오 경로
output_path = "/Users/laxdin24/Downloads/climbing_output.mp4"  # 저장할 결과 비디오 경로

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

frame_index = 0
hold_logs = {}

# 홀드 자동 감지 (색 기반)
def detect_holds(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_orange = np.array([5, 100, 100])
    upper_orange = np.array([20, 255, 255])
    mask = cv2.inRange(hsv, lower_orange, upper_orange)
    mask = cv2.dilate(mask, None, iterations=2)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    regions = []
    for c in contours:
        if cv2.contourArea(c) > 500:
            x, y, w, h = cv2.boundingRect(c)
            regions.append({"x": x, "y": y, "w": w, "h": h})
    return regions

# 분석
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    hold_regions = detect_holds(frame)
    current_holds = []

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            h, w, _ = frame.shape
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            x_px, y_px = int(thumb_tip.x * w), int(thumb_tip.y * h)

            for hold in hold_regions:
                hx, hy, hw, hh = hold["x"], hold["y"], hold["w"], hold["h"]
                if hx <= x_px <= hx + hw and hy <= y_px <= hy + hh:
                    hold_id = f"Hold_{hx}_{hy}"
                    current_holds.append(hold_id)
                    if hold_id not in hold_logs:
                        hold_logs[hold_id] = []
                    hold_logs[hold_id].append(frame_index)

    # 시각화
    for hold in hold_regions:
        cv2.rectangle(frame, (hold["x"], hold["y"]),
                      (hold["x"] + hold["w"], hold["y"] + hold["h"]),
                      (0, 255, 255), 2)
        cv2.putText(frame, "Hold", (hold["x"], hold["y"] - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    for i, hold_id in enumerate(current_holds):
        cv2.putText(frame, f"Holding {hold_id}", (10, frame_height - 30 - i * 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    for i, (hold_id, frames) in enumerate(hold_logs.items()):
        duration = round(len(frames) / fps, 2)
        cv2.putText(frame, f"{hold_id}: {duration}s", (10, 30 + i * 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    out.write(frame)
    frame_index += 1

cap.release()
out.release()
print(f"🎬 비디오 저장 완료: {output_path}")

I0000 00:00:1745541024.274127  824131 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745541024.285184 1314034 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745541024.292734 1314034 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745541042.441014 1314035 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


🎬 비디오 저장 완료: /Users/laxdin24/Downloads/climbing_output.mp4


---

# 자세 추정랜드마크로 진행하기

In [3]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# === 1. 파일 경로 설정 ===
model_path = '/Users/laxdin24/Downloads/pose_landmarker_heavy.task'   # 다운로드한 모델 경로
input_video_path = '/Users/laxdin24/Downloads/IMG_5749.mp4'        # 분석할 클라이밍 영상
output_video_path = '/Users/laxdin24/Downloads/pose_output.mp4'      # 저장할 출력 영상

# === 2. Pose Landmarker 옵션 설정 ===
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=model_path),
    output_segmentation_masks=False,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1
)

# === 3. 모델 초기화 ===
landmarker = vision.PoseLandmarker.create_from_options(options)

# === 4. 입력 영상 열기 ===
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 출력 포맷 지정

# === 5. 출력 비디오 설정 ===
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# === 6. 프레임 처리 ===
timestamp = 0  # 마이크로초 단위

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR → RGB 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    # 포즈 추정 실행
    result = landmarker.detect_for_video(mp_image, timestamp)

    # 결과가 있을 경우 랜드마크 그리기
    if result.pose_landmarks:
        annotated_frame = frame.copy()

        for landmark in result.pose_landmarks[0]:
            x = int(landmark.x * width)
            y = int(landmark.y * height)
            cv2.circle(annotated_frame, (x, y), 5, (0, 255, 0), -1)

        # 출력 영상에 프레임 추가
        out.write(annotated_frame)
    else:
        out.write(frame)  # 실패 시 원본 저장

    # 타임스탬프 업데이트
    timestamp += int(1e6 / fps)

# === 7. 마무리 ===
cap.release()
out.release()
print(f"✅ 자세 추정 영상 저장 완료: {output_video_path}")

I0000 00:00:1745990066.640092 10994525 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1745990066.711158 11036834 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745990066.751309 11036835 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 자세 추정 영상 저장 완료: pose_output.mp4


* 얼굴 제외

In [3]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# 경로 설정
model_path = '/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/Climbing-Project-MakeDataset/pose_landmarker_heavy.task'   # 다운로드한 모델 경로
video_path = '/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/Climbing-Project-MakeDataset/climbvideo/KakaoTalk_Video_2025-05-01-17-05-12.mp4'        # 분석할 클라이밍 영상
output_path = '/Users/laxdin24/Downloads/pose_output.mp4'      # 저장할 출력 영상

# Pose Landmarker 설정
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=model_path),
    output_segmentation_masks=False,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1
)

# 모델 초기화
landmarker = vision.PoseLandmarker.create_from_options(options)

# 비디오 열기
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 랜드마크 연결선 정의 (MediaPipe 문서 기준)
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7),
    (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10), (11, 12),
    (11, 13), (13, 15),
    (12, 14), (14, 16),
    (11, 23), (12, 24),
    (23, 24), (23, 25), (24, 26),
    (25, 27), (26, 28),
    (27, 29), (28, 30),
    (29, 31), (30, 32)
]

timestamp = 0  # 마이크로초 단위

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    result = landmarker.detect_for_video(mp_image, timestamp)

    if result.pose_landmarks:
        annotated_frame = frame.copy()
        lm = result.pose_landmarks[0]

        # ===== 점 그리기 (얼굴 제외) =====
        for idx, point in enumerate(lm):
            if idx <= 10:  # 얼굴 랜드마크는 건너뜀
                continue
            cx = int(point.x * width)
            cy = int(point.y * height)
            cv2.circle(annotated_frame, (cx, cy), 5, (0, 255, 0), -1)

        # ===== 선 그리기 (양쪽 눈/귀/입 등 연결 제외) =====
        for start_idx, end_idx in POSE_CONNECTIONS:
            if start_idx <= 10 or end_idx <= 10:  # 얼굴 연결선 제외
                continue
            x1 = int(lm[start_idx].x * width)
            y1 = int(lm[start_idx].y * height)
            x2 = int(lm[end_idx].x * width)
            y2 = int(lm[end_idx].y * height)
            cv2.line(annotated_frame, (x1, y1), (x2, y2), (255, 255, 255), 2)

        out.write(annotated_frame)
    else:
        out.write(frame)

    timestamp += int(1e6 / fps)

cap.release()
out.release()
print(f"✅ 자세 추정 + 선 연결 완료! 저장 파일: {output_path}")

I0000 00:00:1746160233.311398 1200017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1746160233.380440 1231350 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746160233.417736 1231348 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746160233.540004 1231355 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


✅ 자세 추정 + 선 연결 완료! 저장 파일: /Users/laxdin24/Downloads/pose_output.mp4


## 기존 추정값에서 튀거나 불안정한값 보정하기

In [11]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from collections import deque
import numpy as np

# 경로 설정
model_path = '/Users/laxdin24/Downloads/pose_landmarker_heavy.task'   # 다운로드한 모델 경로
input_video_path = '/Users/laxdin24/Downloads/IMG_5749.mp4'        # 분석할 클라이밍 영상
output_video_path = '/Users/laxdin24/Downloads/pose_output.mp4'      # 저장할 출력 영상

# Pose Landmarker 설정
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=model_path),
    output_segmentation_masks=False,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1
)

# 기본 설정
NUM_LANDMARKS = 33
WINDOW_SIZE = 5
JUMP_THRESHOLD = 0.15  # 관절 간 y 변화 제한

# 다중 프레임을 저장할 deque (최신 N프레임 보관)
frame_history = deque(maxlen=WINDOW_SIZE)

# ==================이동 평균 함수==================
def moving_average(frames, index):
    xs, ys, zs = [], [], []
    for f in frames:
        x, y, z = f[index]
        xs.append(x)
        ys.append(y)
        zs.append(z)
    return np.mean(xs), np.mean(ys), np.mean(zs)

# ==================보정 함수==================
def correct_landmarks(current_frame):
    """
    current_frame: [(x, y, z), ...]  # 현재 프레임의 랜드마크 리스트
    return: 보정된 [(x, y, z), ...]
    """
    frame_history.append(current_frame)
    if len(frame_history) < WINDOW_SIZE:
        return current_frame  # 초기 프레임은 그대로 반환

    # 이전 보정된 프레임
    prev_frame = frame_history[-2]  # 바로 이전 프레임
    corrected_frame = []

    for i in range(NUM_LANDMARKS):
        avg_x, avg_y, avg_z = moving_average(frame_history, i)

        # 점프 제한 적용
        prev_y = prev_frame[i][1]
        if abs(avg_y - prev_y) > JUMP_THRESHOLD:
            avg_y = prev_y  # y값 급변 시 이전 프레임으로 보정

        corrected_frame.append((avg_x, avg_y, avg_z))

    return corrected_frame
# ==============================================

# 모델 초기화
landmarker = vision.PoseLandmarker.create_from_options(options)

# 비디오 열기
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 랜드마크 연결선 정의 (MediaPipe 문서 기준)
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7),
    (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10), (11, 12),
    (11, 13), (13, 15),
    (12, 14), (14, 16),
    (11, 23), (12, 24),
    (23, 24), (23, 25), (24, 26),
    (25, 27), (26, 28),
    (27, 29), (28, 30),
    (29, 31), (30, 32)
]

timestamp = 0  # 마이크로초 단위

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    result = landmarker.detect_for_video(mp_image, timestamp)

    if result.pose_landmarks:
        annotated_frame = frame.copy()
        lm = result.pose_landmarks[0]

         # 📌 좌표만 추출하여 보정 적용
        current_frame_coords = [(pt.x, pt.y, pt.z) for pt in lm]
        corrected_lm = correct_landmarks(current_frame_coords)

        # 보정된 랜드마크 그리기
        # ===== 점 그리기 (얼굴 제외) =====
        for idx, (x, y, z) in enumerate(corrected_lm):
            if idx <= 10:
                continue
            cx = int(x * width)
            cy = int(y * height)
            cv2.circle(annotated_frame, (cx, cy), 5, (0, 255, 0), -1)

        # ===== 선 그리기 (양쪽 눈/귀/입 등 연결 제외) =====
        for start_idx, end_idx in POSE_CONNECTIONS:
            if start_idx <= 10 or end_idx <= 10:
                continue
            x1, y1, _ = corrected_lm[start_idx]
            x2, y2, _ = corrected_lm[end_idx]
            p1 = (int(x1 * width), int(y1 * height))
            p2 = (int(x2 * width), int(y2 * height))
            cv2.line(annotated_frame, p1, p2, (255, 255, 255), 2)

        out.write(annotated_frame)
    else:
        out.write(frame)

    timestamp += int(1e6 / fps)

cap.release()
out.release()
print(f"✅ 자세 추정 + 선 연결 완료! 저장 파일: {output_path}")

I0000 00:00:1745994241.711018 10994525 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1745994241.780034 11163941 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745994241.818666 11163944 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 자세 추정 + 선 연결 완료! 저장 파일: /Users/laxdin24/Downloads/output_landmarked.mp4


## 랜드마크 Json 저장하기
* 1차 기본 랜드마크수치를 보정해서 json 파일화 및 랜드마크 그리기 후 비디오 저장

In [ ]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from collections import deque
import numpy as np

import json  # json 저장용
landmark_json_data = []  # JSON 구조 저장용
frame_index = 0  # 프레임 인덱스 초기화

# 경로 설정
model_path = '/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/Climbing-Project-MakeDataset/pose_landmarker_heavy.task'   # 다운로드한 모델 경로
input_video_path = '/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/Climbing-Project-MakeDataset/climbvideo/KakaoTalk_Video_2025-05-01-17-05-12.mp4'        # 분석할 클라이밍 영상
output_video_path = '/Users/laxdin24/Downloads/pose_output.mp4'      # 저장할 출력 경로

# Pose Landmarker 설정
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=model_path),
    output_segmentation_masks=False,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1
)

# 기본 설정
NUM_LANDMARKS = 33
WINDOW_SIZE = 10
"""

"""

JUMP_THRESHOLD = 0.001  # 관절 간 y 변화 제한
'''
고정 자세 감지 (예: 요가, 정지 상태) : 0.01 ~ 0.03 / 아주 작은 변화만 허용
일반 스포츠 (걷기, 뛰기 등) : 0.05 ~ 0.15 / 자연스러운 움직임 허용
격렬한 동작 (클라이밍, 점프 등) : 0.15 ~ 0.35 / 급격한 y변화도 수용
'''


# 다중 프레임을 저장할 deque (최신 N프레임 보관)
frame_history = deque(maxlen=WINDOW_SIZE)

# ==================이동 평균 함수==================
def moving_average(frames, index):
    xs, ys, zs = [], [], []
    for f in frames:
        x, y, z = f[index]
        xs.append(x)
        ys.append(y)
        zs.append(z)
    return np.mean(xs), np.mean(ys), np.mean(zs)

# ==================보정 함수==================
def correct_landmarks(current_frame):
    """
    current_frame: [(x, y, z), ...]  # 현재 프레임의 랜드마크 리스트
    return: 보정된 [(x, y, z), ...]
    """
    frame_history.append(current_frame)
    if len(frame_history) < WINDOW_SIZE:
        return current_frame  # 초기 프레임은 그대로 반환

    # 이전 보정된 프레임
    prev_frame = frame_history[-2]  # 바로 이전 프레임
    corrected_frame = []

    for i in range(NUM_LANDMARKS):
        avg_x, avg_y, avg_z = moving_average(frame_history, i)

        # 점프 제한 적용
        prev_y = prev_frame[i][1]
        if abs(avg_y - prev_y) > JUMP_THRESHOLD:
            avg_y = prev_y  # y값 급변 시 이전 프레임으로 보정

        corrected_frame.append((avg_x, avg_y, avg_z))

    return corrected_frame
# ==============================================

# 모델 초기화
landmarker = vision.PoseLandmarker.create_from_options(options)

# 비디오 열기
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# 랜드마크 연결선 정의 (MediaPipe 문서 기준)
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7),
    (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10), (11, 12),
    (11, 13), (13, 15),
    (12, 14), (14, 16),
    (11, 23), (12, 24),
    (23, 24), (23, 25), (24, 26),
    (25, 27), (26, 28),
    (27, 29), (28, 30),
    (29, 31), (30, 32)
]

timestamp = 0  # 마이크로초 단위

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    result = landmarker.detect_for_video(mp_image, timestamp)

    if result.pose_landmarks:
        annotated_frame = frame.copy()
        lm = result.pose_landmarks[0]

         # 📌 좌표만 추출하여 보정 적용
        current_frame_coords = [(pt.x, pt.y, pt.z) for pt in lm]
        corrected_lm = correct_landmarks(current_frame_coords)

        # 보정된 랜드마크 그리기
        # ===== 점 그리기 (얼굴 제외, 일정 신뢰도 이상만) =====
        for idx, (x, y, z) in enumerate(corrected_lm):
            if idx <= 10:
                continue
            # if lm[idx].visibility <= 0:
            #     continue  # 신뢰도 낮으면 스킵
            cx = int(x * width)
            cy = int(y * height)
            cv2.circle(annotated_frame, (cx, cy), 5, (0, 255, 0), -1)

        # ===== 선 그리기 (양쪽 눈/귀/입 등 연결 제외, 일정 신뢰도 이상만) =====
        for start_idx, end_idx in POSE_CONNECTIONS:
            if start_idx <= 10 or end_idx <= 10:
                continue
            # if lm[start_idx].visibility <= 0 or lm[end_idx].visibility <= 0:
            #     continue  # 연결점 둘 중 하나라도 신뢰도 낮으면 생략
            x1, y1, _ = corrected_lm[start_idx]
            x2, y2, _ = corrected_lm[end_idx]
            p1 = (int(x1 * width), int(y1 * height))
            p2 = (int(x2 * width), int(y2 * height))
            cv2.line(annotated_frame, p1, p2, (255, 255, 255), 2)

        # ===== frame_index 텍스트 추가 =====
        cv2.putText(
            annotated_frame,
            f"Frame: {frame_index}",
            (20, 40),                     # 위치 (좌측 상단)
            cv2.FONT_HERSHEY_SIMPLEX,     # 글꼴
            1.0,                          # 크기
            (255, 255, 0),                # 색상 (노란색)
            2                            # 두께
        )

        out.write(annotated_frame)

        # === JSON 저장용 구조 만들기 ===
        frame_data = {
            "frame_index": frame_index,
            "landmarks": [
            {
                "landmark_index": idx,
                "x": float(x),
                "y": float(y),
                "z": float(z),
                "visibility": float(lm[idx].visibility)
            }
            for idx, (x, y, z) in enumerate(corrected_lm)
        ]
        }
        landmark_json_data.append(frame_data)
        frame_index += 1
    else:
        out.write(frame)

    timestamp += int(1e6 / fps)

cap.release()
out.release()
print(f"✅ 자세 추정 + 선 연결 완료! 저장 파일: {output_video_path}")
# ===== JSON 파일 저장 =====
json_output_path = '/Users/laxdin24/Downloads/landmarks_output.json'
with open(json_output_path, 'w') as f:
    json.dump(landmark_json_data, f, indent=2)
print(f"✅ 랜드마크 JSON 저장 완료: {json_output_path}")

I0000 00:00:1746160511.641729 1200017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1746160511.710026 1240506 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746160511.748521 1240507 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 자세 추정 + 선 연결 완료! 저장 파일: /Users/laxdin24/Downloads/pose_output.mp4
✅ 랜드마크 JSON 저장 완료: /Users/laxdin24/Downloads/landmarks_output.json



✅ 1. 이동 평균 (Moving Average)
	•	여러 프레임을 평균내어 관절 위치를 부드럽게 보정
	•	단기 노이즈 제거에 효과적

⸻

✅ 2. 축별 점프 제한 (Jump Threshold)
	•	x, y, z 좌표가 이전 프레임과 너무 차이 나면 무시하고 이전 값 유지
	•	갑작스러운 튐 방지

⸻

✅ 3. 저역통과 필터 (Low-pass Filter)
	•	이전 프레임과 선형 혼합 → (alpha * 현재 + (1-alpha) * 이전)
	•	관절의 잔떨림 완화

⸻

✅ 4. Visibility Threshold
	•	관절의 신뢰도(visibility)가 기준 이하일 경우 시각화 제외
	•	잘못 인식된 관절의 시각적 튐 제거

In [8]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import numpy as np
from collections import deque
import json

# ======= 사용자 설정 파라미터 =======
VIDEO_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/Climbing-Project-MakeDataset/climbvideo/KakaoTalk_Video_2025-05-01-17-05-12.mp4"
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/Climbing-Project-MakeDataset/pose_landmarker_heavy.task"
OUTPUT_VIDEO_PATH = "/Users/laxdin24/Downloads/pose_output_filtered.mp4"
OUTPUT_JSON_PATH = "/Users/laxdin24/Downloads/landmarks_output_filtered.json"

WINDOW_SIZE = 6         # 이동 평균 창 크기
JUMP_THRESHOLD = 0.002   # 축별 급변 필터링 기준
VIS_THRESHOLD = 0     # visibility 필터 기준
ALPHA = 0.6             # 1차 저역통과 필터 비율

# ===================================

NUM_LANDMARKS = 33
frame_history = deque(maxlen=WINDOW_SIZE)
prev_filtered_frame = None
landmark_json_data = []
frame_index = 0

# ========== 보정 함수들 ==========
def moving_average(frames, index):
    xs, ys, zs = zip(*[f[index] for f in frames])
    return np.mean(xs), np.mean(ys), np.mean(zs)

def low_pass_filter(new_val, prev_val, alpha=ALPHA):
    if prev_val is None:
        return new_val
    return tuple(alpha * nv + (1 - alpha) * pv for nv, pv in zip(new_val, prev_val))

def correct_landmarks(current_frame):
    frame_history.append(current_frame)
    if len(frame_history) < WINDOW_SIZE:
        return current_frame

    prev_frame = frame_history[-3]
    corrected = []
    global prev_filtered_frame

    for i in range(NUM_LANDMARKS):
        x, y, z = moving_average(frame_history, i)

        # x, y, z 튐 방지
        prev_x, prev_y, prev_z = prev_frame[i]
        if abs(x - prev_x) > JUMP_THRESHOLD:
            x = prev_x
        if abs(y - prev_y) > JUMP_THRESHOLD:
            y = prev_y
        if abs(z - prev_z) > JUMP_THRESHOLD:
            z = prev_z

        filtered = low_pass_filter((x, y, z), prev_filtered_frame[i] if prev_filtered_frame else None)
        corrected.append(filtered)

    prev_filtered_frame = corrected
    return corrected

# ========== MediaPipe 초기화 ==========
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    output_segmentation_masks=False,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1
)
landmarker = vision.PoseLandmarker.create_from_options(options)

cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (w, h))

POSE_CONNECTIONS = [
    (11,12), (11,13), (13,15), (12,14), (14,16),
    (11,23), (12,24), (23,24), (23,25), (24,26),
    (25,27), (26,28), (27,29), (28,30), (29,31), (30,32)
]

timestamp = 0

# ========== 비디오 프레임 반복 ==========
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)
    result = landmarker.detect_for_video(mp_image, timestamp)

    if result.pose_landmarks:
        annotated = frame.copy()
        lm = result.pose_landmarks[0]
        coords = [(pt.x, pt.y, pt.z) for pt in lm]
        filtered = correct_landmarks(coords)

        # 시각화: 점과 선 (얼굴 제외, visibility 무시 or 활용 가능)
        for idx, (x, y, z) in enumerate(filtered):
            if idx <= 10 or lm[idx].visibility < VIS_THRESHOLD:
                continue
            cv2.circle(annotated, (int(x * w), int(y * h)), 4, (0, 255, 0), -1)

        for s, e in POSE_CONNECTIONS:
            if lm[s].visibility < VIS_THRESHOLD or lm[e].visibility < VIS_THRESHOLD:
                continue
            x1, y1, _ = filtered[s]
            x2, y2, _ = filtered[e]
            cv2.line(annotated, (int(x1 * w), int(y1 * h)), (int(x2 * w), int(y2 * h)), (255, 255, 255), 2)

        # 프레임 인덱스 표시
        cv2.putText(annotated, f"Frame: {frame_index}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2)

        out.write(annotated)

        # JSON 저장용
        frame_data = {
            "frame_index": frame_index,
            "landmarks": [
                {
                    "landmark_index": idx,
                    "x": float(x),
                    "y": float(y),
                    "z": float(z),
                    "visibility": float(lm[idx].visibility)
                }
                for idx, (x, y, z) in enumerate(filtered)
            ]
        }
        landmark_json_data.append(frame_data)

    else:
        out.write(frame)

    frame_index += 1
    timestamp += int(1e6 / fps)

cap.release()
out.release()
with open(OUTPUT_JSON_PATH, 'w') as f:
    json.dump(landmark_json_data, f, indent=2)

print(f"🎯 완료: 보정된 포즈 영상 저장 → {OUTPUT_VIDEO_PATH}")
print(f"📝 JSON 저장 완료 → {OUTPUT_JSON_PATH}")

I0000 00:00:1746165756.498871 1200017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1746165756.560631 1394167 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746165756.591581 1394167 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


🎯 완료: 보정된 포즈 영상 저장 → /Users/laxdin24/Downloads/pose_output_filtered.mp4
📝 JSON 저장 완료 → /Users/laxdin24/Downloads/landmarks_output_filtered.json
